In [1]:
# Automated Plant Disease Detection Using Deep Learning and Cloud Deployment



In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt

# Dataset path
data_dir = r"C:\Users\DELL\Desktop\nirupa\Guvi\Final_Project_guvi\archive\plantvillage dataset\color"

# Parameters
img_size = (128, 128)
batch_size = 8
val_split = 0.2
seed = 42

# Train & Validation Split
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=val_split,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=val_split,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

# Class names
class_names = train_ds.class_names
print("Classes:", class_names)

Found 54305 files belonging to 38 classes.
Using 43444 files for training.
Found 54305 files belonging to 38 classes.
Using 10861 files for validation.
Classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato__

In [3]:
from tensorflow.keras import layers

AUTOTUNE = tf.data.AUTOTUNE

# Cache and prefetch for performance
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Data augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.2)
])

In [4]:
from tensorflow.keras import models, layers

baseline_model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(class_names), activation='softmax')
])

baseline_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

C:\Users\DELL\anaconda3\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
epochs = 5
history = baseline_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/5
2535/5431 ━━━━━━━━━━━━━━━━━━━━ 6:59 145ms/step - accuracy: 0.3344 - loss: 2.5161

In [ ]:
baseline_model.save("baseline_model.h5")
print("✅ Model saved as baseline_model.h5")

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model("baseline_model.h5")
model.summary()

In [ ]:
print(len(class_names))
print("Model output units:", baseline_model.output_shape)

In [ ]:
print(baseline_model.summary())

In [ ]:
from tensorflow.keras import layers

# ✅ Data Augmentation Block
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.2)
])


In [ ]:
from tensorflow.keras import Model, layers
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(128,128,3))
base_model.trainable = False  # freeze for transfer learning

inputs = tf.keras.Input(shape=(128,128,3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)

transfer_model = Model(inputs, outputs)

transfer_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_transfer = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

In [ ]:
# Unfreeze some layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-50]:  # keep first ~50 layers frozen
    layer.trainable = False

# Recompile with smaller learning rate
transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train again (fine-tuning stage)
fine_tune_history = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# Save fine-tuned model
transfer_model.save("plant_disease_resnet50_finetuned.keras")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint("best_plant_disease_model.keras", save_best_only=True)
]

fine_tune_history = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns

# Evaluate model
loss, acc = transfer_model.evaluate(val_ds)
print(f"Validation Accuracy: {acc:.2f}")

# Predictions
y_true = np.concatenate([y for x, y in val_ds], axis=0)
y_pred = np.argmax(transfer_model.predict(val_ds), axis=1)

# Reports
print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
last_conv_layer_name = "conv5_block3_out"  # for ResNet50

In [ ]:
pip install tf-keras-vis

In [ ]:
baseline_model(tf.zeros((1, 128, 128, 3)))

In [ ]:
baseline_model.build((None, 128, 128, 3))

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2

# -----------------------------------------------------------
# 1️⃣  Make sure model is built
# -----------------------------------------------------------
_ = baseline_model(tf.zeros((1, 128, 128, 3)))

# -----------------------------------------------------------
# 2️⃣  Pick one image from validation dataset
# -----------------------------------------------------------
for images, labels in val_ds.take(1):
    img_array = images[0:1]
    true_label = labels[0].numpy()

# -----------------------------------------------------------
# 3️⃣  Find the last Conv2D layer automatically
# -----------------------------------------------------------
last_conv_layer = None
for layer in baseline_model.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer = layer
        break
assert last_conv_layer is not None, "No Conv2D layer found in model."

# -----------------------------------------------------------
# 4️⃣  Forward pass & record activations for Grad-CAM
# -----------------------------------------------------------
with tf.GradientTape() as tape:
    # Build intermediate model that outputs conv activations + predictions
    intermediate_model = tf.keras.Model(
        inputs=baseline_model.layers[0].input,
        outputs=[last_conv_layer.output, baseline_model.layers[-1].output]
    )
    conv_outputs, predictions = intermediate_model(img_array)
    pred_index = tf.argmax(predictions[0])
    class_channel = predictions[:, pred_index]

# -----------------------------------------------------------
# 5️⃣  Compute gradients & pooled grads
# -----------------------------------------------------------
grads = tape.gradient(class_channel, conv_outputs)
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
conv_outputs = conv_outputs[0]

# -----------------------------------------------------------
# 6️⃣  Generate Grad-CAM heatmap
# -----------------------------------------------------------
heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
heatmap = np.maximum(heatmap, 0)
if np.max(heatmap) != 0:
    heatmap = heatmap / np.max(heatmap)

# -----------------------------------------------------------
# 7️⃣  Prepare original image for visualization
# -----------------------------------------------------------
img_display = img_array[0].numpy()
if img_display.max() > 1.0:
    img_display = np.clip(img_display / 255.0, 0, 1)
else:
    img_display = np.clip(img_display, 0, 1)

# -----------------------------------------------------------
# 8️⃣  Resize and overlay the heatmap
# -----------------------------------------------------------
heatmap = cv2.resize(heatmap, (img_display.shape[1], img_display.shape[0]))
heatmap = np.uint8(255 * heatmap)
jet = cm.get_cmap("jet")
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]
superimposed_img = np.clip(jet_heatmap * 0.4 + img_display, 0, 1)

# -----------------------------------------------------------
# 9️⃣  Display all results
# -----------------------------------------------------------
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(img_display)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(heatmap, cmap="jet")
plt.title("Grad-CAM Heatmap")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(superimposed_img)
plt.title(f"Overlay → Pred: {class_names[int(pred_index)]}")
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model("baseline_model.h5")
model.summary()